In [1]:
# add mesh4d package folder to path
import sys
sys.path.insert(0, 'D:\\knpob\\20220723-Mesh4D')

# mesh loading settingsx
meshes_path = "D:\\knpob\\20220723-Mesh4D\\data\\meshes\\6kmh_braless_26markers"
start=0
stride = 12
end=120

# fe file path
regist_path = "..\\temp\\regist"
# fe_path = "..\\temp\\c10 init"
# fe_path = "..\\temp\\c10 0.00015"
# fe_path = "..\\temp\\c10 0.0001"
# fe_path = "..\\temp\\c10 0.0002"
fe_path = "..\\temp\\traced"

# Load 4D scanning sequence

In [2]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[====================] 100.0% loading: D:\knpob\20220723-Mesh4D\data\meshes\6kmh_braless_26markers\speed_6km_h_26_marker_set_.000121.obj


In [3]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

# Load breast trace

In [4]:
import os
import numpy as np

breast_trace = np.load(os.path.join(fe_path, 'breast_trace.npy'))
breast_trace.shape

(222, 4854, 3)

In [5]:
rot = np.load(os.path.join(regist_path, 'rot.npy'))
scale = np.load(os.path.join(regist_path, 'scale.npy'))
t = np.load(os.path.join(regist_path, 't.npy'))

In [6]:
plot_num = len(breast_trace)
plot_stride = 20

mesh_num = len(mesh_ls)
mesh_fps = 12
fe_fps = 200

In [7]:
for idx in range(0, plot_num, plot_stride):
    # convert fe index to mesh index
    mesh_idx = int(idx / fe_fps * mesh_fps)

    if mesh_idx >= mesh_num:
        break

    print(idx, mesh_idx)

0 0
20 1
40 2
60 3
80 4
100 6
120 7
140 8
160 9
180 10


In [8]:
# before alignment
import pyvista as pv
from mesh4d import utils
from mesh4d.analyse import measure

scene = pv.Plotter()
scene.open_gif(os.path.join(fe_path, 'breast_trace' + '.gif'))

for idx in range(0, plot_num, plot_stride):
    # convert fe index to mesh index
    mesh_idx = int(idx / fe_fps * mesh_fps)

    if mesh_idx >= mesh_num:
        break

    scene.clear()

    points = breast_trace[idx]
    nearest_points = measure.nearest_points_from_plane(mesh_ls[mesh_idx], points)

    cloud = pv.PolyData(points)
    cloud['point_color'] = np.linalg.norm(nearest_points - points, axis=1)
    
    scene.add_points(cloud, point_size=1e-5, opacity=0.1, cmap='cool')
    scene.add_mesh(mesh_ls[mesh_idx], opacity=0.1, show_edges=True)
    scene.camera_position = 'xy'
    scene.write_frame()

    percent = (idx + 1) / plot_num
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(idx))

scene.close()

[================    ] 81.5% exported the 180-th frame

In [9]:
# after alignment
scene = pv.Plotter()
scene.open_gif(os.path.join(fe_path, 'breast_trace_aligned' + '.gif'))

for idx in range(0, plot_num, plot_stride):
    # convert fe index to mesh index
    mesh_idx = int(idx / fe_fps * mesh_fps)

    if mesh_idx >= mesh_num:
        break

    scene.clear()

    points = breast_trace[idx]
    aligned_points = (scale * np.matmul(rot, points.T)).T + t
    nearest_points = measure.nearest_points_from_plane(mesh_ls[mesh_idx], aligned_points)

    cloud = pv.PolyData(aligned_points)
    cloud['point_color'] = np.linalg.norm(nearest_points - aligned_points, axis=1)
    
    scene.add_points(cloud, point_size=1e-5, opacity=0.1, cmap='cool')
    scene.add_mesh(mesh_ls[mesh_idx], opacity=0.1, show_edges=True)
    scene.camera_position = 'xy'
    scene.write_frame()

    percent = (idx + 1) / plot_num
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(idx))

scene.close()

[================    ] 81.5% exported the 180-th frame